In [1]:
import os
import sys

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
def open_df_title_unit_csv(file_path):

    # Read the file and store comments
    with open(file_path, 'r') as file:
        lines = file.readlines()

    comments = [line for line in lines if line.startswith('#')]
    data = [line for line in lines if not line.startswith('#')]

    # # Extract the title and unit from comments
    title_index = comments.index(next(line for line in comments if line.startswith('## Title') or line.startswith('## File content')))
    title = comments[title_index + 1].strip()
    title = title.replace('#', '').strip()
    # Extract the unit from comments if there is formated like this: 
    """
    ## Unit
    ### MWh
    """
    unit_index = comments.index(next(line for line in comments if line.startswith('## Unit')))
    unit = comments[unit_index + 1].strip()
    unit = unit.replace('#', '').strip()

    # Write the data back to a temporary file
    temp_file_path = file_path + '.tmp'
    with open(temp_file_path, 'w') as temp_file:
        temp_file.writelines(data)

    # Display the comments
    # for comment in comments:
    #     print(comment.strip())

    # Read the data with pandas
    df = pd.read_csv(temp_file_path, index_col='Date')

    # Optionally, remove the temporary file
    os.remove(temp_file_path)
    
    return df, title, unit

In [3]:
super_directory = r"..\..\wattwise\data"
directory1 = r"..\..\wattwise\data\meteo"
directory2 = r"..\..\wattwise\data\energy"

In [4]:
df_es = pd.DataFrame()

for directory in os.listdir(super_directory):
    print(f"Processing directory: {directory}")
    directory_path = os.path.join(super_directory, directory)
    print(f"Processing directory: {directory_path}")
    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".csv"):
            try:
                file_path = os.path.join(directory_path, filename)
                print(f"Reading file: {file_path}")
                
                df, title, unit = open_df_title_unit_csv(file_path)
                print(f"Title: {title}")
                title_parts = title.split()
                if len(title_parts) > 5:
                    title = ' '.join(title_parts[:2])
                print(f"Title after processing: {title}")
                print(f"Unit: {unit}")
                
                # Ensure the dataset contains an 'ES' column to filter for Spain
                if 'ES' in df.columns:
                    # Keep only the 'Date' and 'ES' columns
                    df = df[['ES']].copy()
                    df.reset_index(inplace=True)
                else:
                    print(f"Skipping {filename} as no 'ES' column found.")
                    continue
                
                
                # Convert 'Date' column to datetime format before merging
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                
                # Drop rows where 'Date' could not be converted (if any)
                df = df.dropna(subset=['Date'])
                
                
                # Rename the 'ES' column with the title name
                df.rename(columns={'ES': title}, inplace=True)
                
                # Merge into main DataFrame (on 'Date')
                if df_es.empty:
                    df_es = df  # First dataset initializes df_es
                else:
                    df_es = pd.merge(df_es, df, on='Date', how='outer')  # Merge on 'Date'
                
            except Exception as e:
                print(f"Error processing {filename}: {e}")
        
# Filter data to keep only records from 2020 onwards
df_es = df_es[df_es['Date'] >= '1980-01-01']

# Display rows where date conversion failed (if any remain after dropping NaNs)
invalid_dates = df_es[df_es['Date'].isna()]
if not invalid_dates.empty:
    print("Warning: Some date values could not be converted:")
    print(invalid_dates)
    
# Display the final DataFrame structure
display(df_es.head())

#Save the final DataFrame to a CSV file
df_es.to_csv('spain_energy_meteo_data.csv', index=False)

Processing directory: energy
Processing directory: ..\..\wattwise\data\energy
Reading file: ..\..\wattwise\data\energy\H_ERA5_ECMW_T639_EDM_NA---_Euro_NUT0_S197901010000_E202502282300_NRG_TIM_01d_NA-_noc_org_NA_NA---_NA---_GamNT.csv
Title: Electricity Demand (EDM), expressed as Energy (NRG)
Title after processing: Electricity Demand
Unit: MWh
Reading file: ..\..\wattwise\data\energy\H_ERA5_ECMW_T639_HRE_NA---_Euro_NUT0_S197901010000_E202502282300_NRG_TIM_01d_NA-_noc_org_NA_NA---_NA---_StRnF.csv
Title: Hydropower reservoir (HRE), expressed as Daily Energy (NRG)
Title after processing: Hydropower reservoir
Unit: MWh
Reading file: ..\..\wattwise\data\energy\H_ERA5_ECMW_T639_HRO_NA---_Euro_NUT0_S197901010000_E202502282300_NRG_TIM_01d_NA-_noc_org_NA_NA---_NA---_StRnF.csv
Title: Hydropower run-of-river (HRO), expressed as Daily Energy (NRG)
Title after processing: Hydropower run-of-river
Unit: MWh
Reading file: ..\..\wattwise\data\energy\H_ERA5_ECMW_T639_SPV_0000m_Euro_NUT0_S197901010000_E20

,Date,Electricity Demand,Hydropower reservoir,Hydropower run-of-river,Solar PV Power,Wind Power Onshore,Global Horizontal Irradiance,Mean Sea Level Pressure,Air Temperature,Total Precipitation,Wind Speed_x,Wind Speed_y
365,1980-01-01,555442.4,75105.17,27625.16,7414.011781,241681.204853,63.712332,101892.963958,282.782622,0.001628,3.967427,6.825222
366,1980-01-02,668316.5,74774.20,27676.85,9667.063190,141812.976918,76.356610,102279.375083,280.568923,0.000416,3.075484,5.265852
367,1980-01-03,692024.2,74066.41,28220.82,14965.925374,28256.394755,103.318422,102547.714417,277.943976,0.000005,1.731207,2.884409
368,1980-01-04,693162.4,74689.07,28344.47,14105.305812,36945.997648,99.630959,102265.270542,278.233487,0.000728,1.816051,3.011657
369,1980-01-05,624292.7,74882.60,28480.01,11298.820055,72234.129754,84.680324,102225.539208,278.408883,0.000794,2.186170,3.756692
